In [1]:
from google.colab import drive 

drive.mount('/content/drive/')

!pip install konlpy

import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tqdm import tqdm

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv', encoding = 'utf-8')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 448 kB 44.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
train = train.dropna(how = 'any') # missing value 처리
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [3]:
okt = Okt() # 객체생성

In [4]:
X_train = []
for sentence,i in zip(train['data'],tqdm(range(len(train['data'])))) : # tqdm => 작업 진행률 나타내는 것
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # Text를 형태소 단위로 나눈다
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X) # List 원소 마지막에 추가

100%|█████████▉| 39991/39992 [37:11<00:00, 17.92it/s]


In [5]:
X_test = []
for sentence in test['data']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # Text를 형태소 단위로 나눈다
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

In [6]:
tokenizer = Tokenizer() #  
tokenizer.fit_on_texts(X_train)

In [7]:
vocab_size = 30000
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [8]:
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [9]:
y_train = to_categorical(train['category'])

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

y_pred = model.predict(X_test)

sample_submission['category'] = y_pred
sample_submission.to_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv', encoding='utf-8', index = False)

Epoch 1/15
313/313 [==============================] - 42s 107ms/step - loss: 0.5644 - acc: 0.7860
Epoch 2/15
313/313 [==============================] - 34s 107ms/step - loss: 0.3717 - acc: 0.8766
Epoch 3/15
313/313 [==============================] - 33s 106ms/step - loss: 0.2593 - acc: 0.9115
Epoch 4/15
313/313 [==============================] - 34s 107ms/step - loss: 0.2284 - acc: 0.9206
Epoch 5/15
313/313 [==============================] - 33s 106ms/step - loss: 0.1814 - acc: 0.9372
Epoch 6/15
313/313 [==============================] - 33s 106ms/step - loss: 0.1669 - acc: 0.9426
Epoch 7/15
313/313 [==============================] - 33s 106ms/step - loss: 0.1898 - acc: 0.9338
Epoch 8/15
313/313 [==============================] - 33s 107ms/step - loss: 0.1409 - acc: 0.9522
Epoch 9/15
313/313 [==============================] - 33s 106ms/step - loss: 0.1104 - acc: 0.9615
Epoch 10/15
313/313 [==============================] - 33s 107ms/step - loss: 0.1101 - acc: 0.9610
Epoch 11/15
313/313

In [11]:
sample_submission['category'] = np.argmax(y_pred, axis=-1)
sample_submission.to_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv', encoding='utf-8', index = False)